In [180]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [181]:
data = load_iris()

In [182]:
X = data.data
y = data.target
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1)
Y_train_encoded = OneHotEncoder().fit_transform(Y_train.reshape(-1, 1)).toarray()
Y_test_encoded = OneHotEncoder().fit_transform(Y_test.reshape(-1, 1)).toarray()


n0, m_total = X.shape
print(X_train.shape)
print(X_test.shape)
print(Y_train_encoded.shape)
print(Y_test_encoded.shape)

(120, 4)
(30, 4)
(120, 3)
(30, 3)


In [183]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(Y_train_encoded.shape[1], activation='softmax'))
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 10)                50        
                                                                 
 dense_44 (Dense)            (None, 3)                 33        
                                                                 
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________


In [184]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003), loss='categorical_crossentropy')
history = model.fit(X_train, Y_train_encoded, epochs=100, validation_data=(X_test, Y_test_encoded))

Epoch 1/100
4/4 [==============================] - 0s 39ms/step - loss: 1.3482 - val_loss: 1.2239
Epoch 2/100
4/4 [==============================] - 0s 8ms/step - loss: 1.2281 - val_loss: 1.1856
Epoch 3/100
4/4 [==============================] - 0s 9ms/step - loss: 1.1510 - val_loss: 1.1537
Epoch 4/100
4/4 [==============================] - 0s 9ms/step - loss: 1.0905 - val_loss: 1.0944
Epoch 5/100
4/4 [==============================] - 0s 10ms/step - loss: 1.0431 - val_loss: 1.0401
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 0.9962 - val_loss: 0.9856
Epoch 7/100
4/4 [==============================] - 0s 11ms/step - loss: 0.9553 - val_loss: 0.9435
Epoch 8/100
4/4 [==============================] - 0s 11ms/step - loss: 0.9144 - val_loss: 0.9041
Epoch 9/100
4/4 [==============================] - 0s 9ms/step - loss: 0.8775 - val_loss: 0.8683
Epoch 10/100
4/4 [==============================] - 0s 11ms/step - loss: 0.8422 - val_loss: 0.8277
Epoch 11/100
4/4 [=====

In [185]:
y_pred = np.argmax(model.predict(X_train), axis=1)
print('train accuracy:', accuracy_score(Y_train, y_pred))
y_pred = np.argmax(model.predict(X_test), axis=1)
print('test accuracy:', accuracy_score(Y_test, y_pred))

4/4 [==============================] - 0s 2ms/step
train accuracy: 0.975
1/1 [==============================] - 0s 17ms/step
test accuracy: 0.9666666666666667


For comparison, use logistic regression provided by sklearn to predict result.

In [186]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
X = data.data
y = data.target
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1)
clf.fit(X_train, Y_train.flatten())

LogisticRegression()

In [187]:
y_pred = clf.predict(X_train)
print('train accuracy:', accuracy_score(Y_train, y_pred))
y_pred = clf.predict(X_test)
print('test accuracy:', accuracy_score(Y_test, y_pred))

train accuracy: 0.9833333333333333
test accuracy: 0.9666666666666667
